<a href="https://colab.research.google.com/github/somrajmaji/Swiggy_chatbot/blob/main/Swiggy_Chatbot_NLTK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O intents.json https://techlearn-cdn.s3.amazonaws.com/bs_swiggy_chatbot/intent.json

--2021-09-07 04:55:58--  https://techlearn-cdn.s3.amazonaws.com/bs_swiggy_chatbot/intent.json
Resolving techlearn-cdn.s3.amazonaws.com (techlearn-cdn.s3.amazonaws.com)... 52.219.64.40
Connecting to techlearn-cdn.s3.amazonaws.com (techlearn-cdn.s3.amazonaws.com)|52.219.64.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4699 (4.6K) [application/json]
Saving to: ‘intents.json’

intents.json        100%[===================>]   4.59K  --.-KB/s    in 0s      

2021-09-07 04:55:58 (207 MB/s) - ‘intents.json’ saved [4699/4699]



In [2]:
import json
with open('intents.json', 'r') as file:
  intents = json.load(file)

In [3]:
data=[]
tag = []
response=[]

for intent in intents['intents']:
  for pattern in intent['patterns']:
    data.append(pattern)
    tag.append(intent['tag'])
    response.append(intent['responses'])

In [7]:
import nltk
import numpy as np

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [11]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [12]:
def clean_corpus(corpus):
  # lowering every word in text
  corpus = [ doc.lower() for doc in corpus]
  cleaned_corpus = []
  
  stop_words = stopwords.words('english')
  wordnet_lemmatizer = WordNetLemmatizer()

  # iterating over every text
  for doc in corpus:
    # tokenizing text
    tokens = word_tokenize(doc)
    cleaned_sentence = [] 
    for token in tokens: 
      # removing stopwords, and punctuation
      if token not in stop_words and token.isalpha(): 
        # applying lemmatization
        cleaned_sentence.append(wordnet_lemmatizer.lemmatize(token)) 
    cleaned_corpus.append(' '.join(cleaned_sentence))
  return cleaned_corpus

In [ ]:
corpus = clean_corpus(data)
corpus

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

In [16]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
y = encoder.fit_transform(np.array(tag).reshape(-1,1))

In [ ]:
X.shape

In [ ]:
y.shape

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD



In [32]:
model = Sequential([
                    Dense(128, input_shape=(X.shape[1],), activation='relu'),
                    Dropout(0.2),
                    Dense(64, activation='relu'),
                    Dropout(0.2),
                    Dense(y.shape[1], activation='softmax')
])

In [33]:
model.summary()





Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 128)               5888      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 8)                 520       
Total params: 14,664
Trainable params: 14,664
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(optimizer="adam",loss=categorical_crossentropy,metrics=["acc"])

In [36]:
history = model.fit(X.toarray(), y.toarray(), epochs=20, batch_size=1,validation_split=0.2)

Epoch 1/20
32/32 [==============================] - 0s 8ms/step - loss: 0.1768 - acc: 0.9375 - val_loss: 0.0202 - val_acc: 1.0000
Epoch 2/20
32/32 [==============================] - 0s 2ms/step - loss: 0.1100 - acc: 0.9688 - val_loss: 0.0223 - val_acc: 1.0000
Epoch 3/20
32/32 [==============================] - 0s 2ms/step - loss: 0.1053 - acc: 0.9688 - val_loss: 0.0236 - val_acc: 1.0000
Epoch 4/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0879 - acc: 0.9688 - val_loss: 0.0258 - val_acc: 1.0000
Epoch 5/20
32/32 [==============================] - 0s 2ms/step - loss: 0.0725 - acc: 1.0000 - val_loss: 0.0252 - val_acc: 1.0000
Epoch 6/20
32/32 [==============================] - 0s 2ms/step - loss: 0.1548 - acc: 0.9375 - val_loss: 0.0303 - val_acc: 1.0000
Epoch 7/20
32/32 [==============================] - 0s 2ms/step - loss: 0.1093 - acc: 0.9375 - val_loss: 0.0350 - val_acc: 1.0000
Epoch 8/20
32/32 [==============================] - 0s 3ms/step - loss: 0.0727 - acc: 1.00

In [42]:
INTENT_NOT_FOUND_THRESHOLD = 0.40

def predict_intent_tag(message):
  message = clean_corpus([message])
  X_test = vectorizer.transform(message)
  y = model.predict(X_test.toarray())
  
  # if probability of all intent is low, classify it as noanswer
  if y.max() < INTENT_NOT_FOUND_THRESHOLD:
    return 'noanswer'
  
  prediction = np.zeros_like(y[0])
  prediction[y.argmax()] = 1
  tag = encoder.inverse_transform([prediction])[0][0]
  return tag

In [44]:
print(predict_intent_tag('How you could help me?'))
print(predict_intent_tag('Where\'s my order'))

options
order-status-request


In [45]:
import random
import time 

In [46]:
def get_intent(tag):
  # to return complete intent from intent tag
  for intent in intents['intents']:
    if intent['tag'] == tag:
      return intent

In [47]:
def perform_action(action_code, intent):
  # funition to perform an action which is required by intent
  
  if action_code == 'CHECK_ORDER_STATUS':
    print('\n Checking database \n')
    time.sleep(2)
    order_status = ['in kitchen', 'with delivery executive']
    delivery_time = []
    return {'intent-tag':intent['next-intent-tag'][0],
            'order_status': random.choice(order_status),
            'delivery_time': random.randint(10, 30)}
  
  elif action_code == 'ORDER_CANCEL_CONFIRMATION':
    ch = input('BOT: Do you want to continue (Y/n) ?')
    if ch == 'y' or ch == 'Y':
      choice = 0
    else:
      choice = 1
    return {'intent-tag':intent['next-intent-tag'][choice]}
  
  elif action_code == 'ADD_DELIVERY_INSTRUCTIONS':
    instructions = input('Your Instructions: ')
    return {'intent-tag':intent['next-intent-tag'][0]}

In [48]:
while True:
  # get message from user
  message = input('You: ')
  # predict intent tag using trained neural network
  tag = predict_intent_tag(message)
  # get complete intent from intent tag
  intent = get_intent(tag)
  # generate random response from intent
  response = random.choice(intent['responses'])
  print('Bot: ', response)

  # check if there's a need to perform some action
  if 'action' in intent.keys():
    action_code = intent['action']
    # perform action
    data = perform_action(action_code, intent)
    # get follow up intent after performing action
    followup_intent = get_intent(data['intent-tag'])
    # generate random response from follow up intent
    response = random.choice(followup_intent['responses'])

    # print randomly selected response
    if len(data.keys()) > 1:
      print('Bot: ', response.format(**data))
    else:
      print('Bot: ', response)

  # break loop if intent was goodbye
  if tag == 'goodbye':
    break

You: Hi
Bot:  Good to see you again
You: Order status
Bot:  I am checking up with for your order

 Checking database 

Bot:  Your order will be delivered in 11, currently it is in kitchen. Is there anything else, we can help you with?
You: cancel my order
Bot:  I can cancel your order, If your order is in kitchen, there will be a cancellation fee.  Do you want me to proceed?
BOT: Do you want to continue (Y/n) ?Y
Bot:  I have canceled your order, the order refund will reflect in your account in 2 days. Do you need any more help?
You: no
Bot:  Have a nice day
